In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [2]:
train_df = pd.read_csv('train_Wc8LBpr.csv')
test_df = pd.read_csv('test_VsU9xXK.csv')
train_df.shape,test_df.shape

((131662, 14), (87395, 13))

In [3]:
df = train_df.append(test_df)
df_sub = pd.read_csv('sample_submission_NoPBkjr.csv')
train_df.shape,test_df.shape,df.shape

((131662, 14), (87395, 13), (219057, 14))

In [5]:
# df['Tot_score'] =  df['Location_Score']+df['Internal_Audit_Score']+df['External_Audit_Score']+df['Fin_Score']+df['Loss_score']
# df['Audit_score'] = df['Internal_Audit_Score']+df['External_Audit_Score']
# df['Audit_score_diff'] = abs(df['Internal_Audit_Score']-df['External_Audit_Score'])
# df['Audit_score_bool'] = (df['Internal_Audit_Score']>df['External_Audit_Score'])*1
# df['var1'] = df['Location_Score']/df['Internal_Audit_Score']
# df['var2'] = df['Location_Score']/df['External_Audit_Score']
# df['var3'] = df['Location_Score']/df['Fin_Score']
# df['var4'] = df['Location_Score']/df['Loss_score']

In [4]:
df.isnull().sum()

Trip_ID                             0
Trip_Distance                       0
Type_of_Cab                     33368
Customer_Since_Months            9886
Life_Style_Index                33520
Confidence_Life_Style_Index     33520
Destination_Type                    0
Customer_Rating                     0
Cancellation_Last_1Month            0
Var1                           117819
Var2                                0
Var3                                0
Gender                              0
Surge_Pricing_Type              87395
dtype: int64

In [5]:
df[['Customer_Since_Months','Life_Style_Index','Confidence_Life_Style_Index','Var1']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219057 entries, 0 to 87394
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Customer_Since_Months        209171 non-null  float64
 1   Life_Style_Index             185537 non-null  float64
 2   Confidence_Life_Style_Index  185537 non-null  object 
 3   Var1                         101238 non-null  float64
dtypes: float64(3), object(1)
memory usage: 8.4+ MB


In [6]:
df['Confidence_Life_Style_Index'].value_counts()

B    67265
C    59736
A    58536
Name: Confidence_Life_Style_Index, dtype: int64

In [7]:
for x in ['Customer_Since_Months','Life_Style_Index','Var1']:
    df[x] = df[x].fillna(df[x].mean())
df['Type_of_Cab'] = df['Type_of_Cab'].fillna('Miss')
df['Confidence_Life_Style_Index'] = df['Confidence_Life_Style_Index'].fillna('Miss')

In [8]:
df.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                        0
Customer_Since_Months              0
Life_Style_Index                   0
Confidence_Life_Style_Index        0
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                               0
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type             87395
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219057 entries, 0 to 87394
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Trip_ID                      219057 non-null  object 
 1   Trip_Distance                219057 non-null  float64
 2   Type_of_Cab                  219057 non-null  object 
 3   Customer_Since_Months        219057 non-null  float64
 4   Life_Style_Index             219057 non-null  float64
 5   Confidence_Life_Style_Index  219057 non-null  object 
 6   Destination_Type             219057 non-null  object 
 7   Customer_Rating              219057 non-null  float64
 8   Cancellation_Last_1Month     219057 non-null  int64  
 9   Var1                         219057 non-null  float64
 10  Var2                         219057 non-null  int64  
 11  Var3                         219057 non-null  int64  
 12  Gender                       219057 non-null  object 
 13  

In [10]:
del df['Trip_ID']
df = pd.get_dummies(df)
df.shape

(219057, 35)

In [11]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = MinMaxScaler()
df_1 = scaler.fit_transform(df.drop(['Surge_Pricing_Type'],axis=1))
df_1 = pd.DataFrame(df_1)
df_1.columns = df.drop(['Surge_Pricing_Type'],axis=1).columns
df_1['target'] = list(df['Surge_Pricing_Type'])
df_1.head()


,Trip_Distance,Customer_Since_Months,Life_Style_Index,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Type_of_Cab_A,Type_of_Cab_B,...,Destination_Type_H,Destination_Type_I,Destination_Type_J,Destination_Type_K,Destination_Type_L,Destination_Type_M,Destination_Type_N,Gender_Female,Gender_Male,target
0,0.059310,0.1,0.311993,0.780945,0.00,0.055556,0.071429,0.051948,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
1,0.267719,1.0,0.411721,0.689922,0.00,0.044444,0.190476,0.168831,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
2,0.378902,1.0,0.417384,0.700175,0.25,0.189422,0.190476,0.162338,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
3,0.562339,1.0,0.417384,0.690673,0.00,0.189422,0.142857,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
4,0.501653,1.0,0.482585,0.680420,0.50,0.116667,0.107143,0.324675,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [12]:
X = df_1[df_1.target.notnull()].drop(['target'],axis=1)
y = df_1.target[df_1.target.notnull()]


X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
ml_model = LGBMClassifier(subsample= 0.9,
 reg_lambda= 2.5,
 reg_alpha= 1,
 random_state= 108,
 objective= 'multiclass',
 n_estimators= 4000,
 min_split_gain= 0.5,
 min_data_in_leaf= 20,
 metric= 'multi_logloss',
 max_depth= 8,
 learning_rate= 0.01,
 colsample_bytree= 1,
 boosting_type= 'gbdt')
test_pred_all = []
lst = []
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=108)
# X is the feature set and y is the target
for train_index, test_index in skf.split(X,y): 
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    ml_model = ml_model.fit(X_train, y_train)
    
   
    
    pred = ml_model.predict(X_test)
    print(accuracy_score(y_test,pred))
    lst.append(accuracy_score(y_test,pred))
    test_pred = ml_model.predict(df_1[df_1.target.isnull()].drop(['target'],axis=1))
    test_pred_all.append(test_pred)
print('#### avg log_loss ####')
print(sum(lst)/len(lst))



## 0.2989202402351237  -- 0.27494250876163917


/home/rakesh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.7040328092959672
0.7047922837396522
0.7053015342548989
0.7098587270241531
0.6994531368676895
0.70370651678566
0.7045420021266899


In [ ]:
## lgbm
kk = pd.DataFrame(test_pred_all).T
kk.head()

# kk['ss'] = kk.apply(lambda x : x.value_counts().idxmax() , axis = 1)
# test_df['Surge_Pricing_Type'] = kk['ss']
# test_df[['Trip_ID','Surge_Pricing_Type']].to_csv('lgbm_sub.csv',index=False)

In [ ]:
kk['ss'] = kk.apply(lambda x : x.value_counts().idxmax() , axis = 1)

In [ ]:
kk.tail()

In [ ]:

test_df['Surge_Pricing_Type'] = kk['ss']
test_df[['Trip_ID','Surge_Pricing_Type']].to_csv('lgbm_sub.csv',index=False)

### Random Forest

In [62]:
from sklearn.ensemble import RandomForestClassifier
ml_model = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2, max_features = 20, max_depth = 6, criterion= 'entropy',random_state=108)

test_pred_all = []
lst = []
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=108)
# X is the feature set and y is the target
for train_index, test_index in skf.split(X,y): 
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    ml_model = ml_model.fit(X_train, y_train)
    
   
    
    pred = ml_model.predict(X_test)
    print(accuracy_score(y_test,pred))
    lst.append(accuracy_score(y_test,pred))
    test_pred = ml_model.predict(df_1[df_1.target.isnull()].drop(['target'],axis=1))
    test_pred_all.append(test_pred)
print('#### avg log_loss ####')
print(sum(lst)/len(lst))



## 0.27776065675905426


C:\Users\503111508\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.6801093643198907
0.6823118402065771
0.6792495822573295
0.6903387513291812
0.6805407868752849
0.6854777457086435
0.6895032659881513
0.6923135348625247
0.6771988455111652
0.6835029621753
#### avg log_loss ####
0.6840546679234047


In [63]:
## rf

kk = pd.DataFrame(test_pred_all).T


kk['ss'] = kk.apply(lambda x : x.value_counts().idxmax() , axis = 1)
test_df['Surge_Pricing_Type'] = kk['ss']
test_df[['Trip_ID','Surge_Pricing_Type']].to_csv('rf_sub.csv',index=False)

kk.head()

,0,1,2,3,4,5,6,7,8,9,ss
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


### XGBOOST

In [ ]:
import xgboost as xgb
ml_model = xgb.XGBClassifier(max_depth=30,learning_rate=0.01,colsample_bytree=.9,reg_alpha=3.2, reg_lambda=3.5,n_estimators=3000,gamma=1,n_jobs=-1)
test_pred_all = []
lst = []
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=108)
# X is the feature set and y is the target
for train_index, test_index in skf.split(X,y): 
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    ml_model = ml_model.fit(X_train, y_train)
    
   
    
    pred = ml_model.predict(X_test)
    print(accuracy_score(y_test,pred))
    lst.append(accuracy_score(y_test,pred))
    test_pred = ml_model.predict(df_1[df_1.target.isnull()].drop(['target'],axis=1))
    test_pred_all.append(test_pred)
print('#### avg log_loss ####')
print(sum(lst)/len(lst))




C:\Users\503111508\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
## xgb

kk = pd.DataFrame(test_pred_all).T


kk['ss'] = kk.apply(lambda x : x.value_counts().idxmax() , axis = 1)
test_df['Surge_Pricing_Type'] = kk['ss']
test_df[['Trip_ID','Surge_Pricing_Type']].to_csv('rf_xgb.csv',index=False)

kk.head()

### Final Prediction

In [64]:
lgbm = pd.read_excel('lgbm_final.xlsx')[1]
rf = pd.read_excel('rf_final.xlsx')[1]
xgb = pd.read_excel('xgb_final.xlsx')[1]


In [69]:
fnl = pd.concat([lgbm,rf,xgb],axis=1)
fnl.columns = ['lgbm','rf','xgb']
fnl[1] = fnl['lgbm']*.35+fnl['xgb']*.35 + fnl['rf']*.3
fnl[0] = 1- fnl[1]
fnl_submission = fnl[[0,1]]
fnl_submission

,0,1
0,0.010887,0.989113
1,0.024250,0.975750
2,0.884820,0.115180
3,0.010011,0.989989
4,0.370040,0.629960
...,...,...
228,0.905825,0.094175
229,0.416092,0.583908
230,0.626688,0.373312
231,0.010144,0.989856


In [70]:
fnl_submission.to_excel('fnl_submission.xlsx',index=False)